In [5]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [6]:
adeft_grounder = AdeftGrounder()

In [7]:
shortforms = ['AHR']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [8]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [9]:
list(zip(longforms, counts))

[('aryl hydrocarbon receptor', 1240),
 ('airway hyperresponsiveness', 1216),
 ('airway hyperreactivity', 295),
 ('airway hyper responsiveness', 224),
 ('airway hyper reactivity', 59),
 ('aromatic hydrocarbon receptor', 33),
 ('ah receptor', 27),
 ('and hyperresponsiveness', 18),
 ('arylhydrocarbon receptor', 12),
 ('acute humoral rejection', 8),
 ('aryl hydrocarbon', 7),
 ('acute haemodynamic responsiveness', 6),
 ('adjusted hazard ratio', 5),
 ('airway hyperactivity', 5),
 ('airway inflammation hyperresponsiveness', 5),
 ('antiseptic hand rubbing', 3),
 ('anti hydroxyl radical', 3),
 ('autonomic hyperreflexia', 3),
 ('acute hemodynamic responsiveness', 3),
 ('and hyper responsiveness', 3)]

In [10]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-08 00:11:28] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [11]:
result = [grounding_map, names, pos_labels]

In [12]:
result

[{'acute haemodynamic responsiveness': 'ungrounded',
  'acute hemodynamic responsiveness': 'ungrounded',
  'acute humoral rejection': 'ungrounded',
  'adjusted hazard ratio': 'ungrounded',
  'ah receptor': 'HGNC:348',
  'airway hyper reactivity': 'MESH:D012130',
  'airway hyper responsiveness': 'MESH:D012130',
  'airway hyperactivity': 'MESH:D012130',
  'airway hyperreactivity': 'MESH:D012130',
  'airway hyperresponsiveness': 'MESH:D012130',
  'airway inflammation hyperresponsiveness': 'MESH:D012130',
  'and hyper responsiveness': 'MESH:D012130',
  'and hyperresponsiveness': 'MESH:D012130',
  'anti hydroxyl radical': 'CHEBI:CHEBI:29191',
  'antiseptic hand rubbing': 'ungrounded',
  'aromatic hydrocarbon receptor': 'HGNC:348',
  'aryl hydrocarbon': 'HGNC:348',
  'aryl hydrocarbon receptor': 'HGNC:348',
  'arylhydrocarbon receptor': 'HGNC:348',
  'autonomic hyperreflexia': 'MESH:D020211'},
 {'HGNC:348': 'AHR',
  'MESH:D012130': 'Respiratory Hypersensitivity',
  'CHEBI:CHEBI:29191': 'hydr

In [13]:
grounding_map, names, pos_labels = [{'acute haemodynamic responsiveness': 'ungrounded',
  'acute hemodynamic responsiveness': 'ungrounded',
  'acute humoral rejection': 'ungrounded',
  'adjusted hazard ratio': 'ungrounded',
  'ah receptor': 'HGNC:348',
  'airway hyper reactivity': 'MESH:D012130',
  'airway hyper responsiveness': 'MESH:D012130',
  'airway hyperactivity': 'MESH:D012130',
  'airway hyperreactivity': 'MESH:D012130',
  'airway hyperresponsiveness': 'MESH:D012130',
  'airway inflammation hyperresponsiveness': 'MESH:D012130',
  'and hyper responsiveness': 'MESH:D012130',
  'and hyperresponsiveness': 'MESH:D012130',
  'anti hydroxyl radical': 'CHEBI:CHEBI:29191',
  'antiseptic hand rubbing': 'ungrounded',
  'aromatic hydrocarbon receptor': 'HGNC:348',
  'aryl hydrocarbon': 'HGNC:348',
  'aryl hydrocarbon receptor': 'HGNC:348',
  'arylhydrocarbon receptor': 'HGNC:348',
  'autonomic hyperreflexia': 'MESH:D020211'},
 {'HGNC:348': 'AHR',
  'MESH:D012130': 'Respiratory Hypersensitivity',
  'CHEBI:CHEBI:29191': 'hydroxyl',
  'MESH:D020211': 'Autonomic Dysreflexia'},
 ['HGNC:348', 'MESH:D012130']]

In [14]:
excluded_longforms = []

In [15]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [16]:
additional_entities = {}

In [17]:
unambiguous_agent_texts = {}

In [18]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [19]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [20]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[('HGNC:17981', 'HGNC:17981', 29)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [21]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-08 00:22:18] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-08 00:22:50] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9950883841339072 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [22]:
classifier.stats

{'label_distribution': {'MESH:D012130': 1385,
  'HGNC:348': 952,
  'ungrounded': 16,
  'MESH:D020211': 3,
  'CHEBI:CHEBI:29191': 3},
 'f1': {'mean': 0.995088, 'std': 0.001091},
 'precision': {'mean': 0.993624, 'std': 0.00133},
 'recall': {'mean': 0.996577, 'std': 0.001713},
 'MESH:D012130': {'f1': {'mean': 0.996396, 'std': 0.00161},
  'pr': {'mean': 0.997834, 'std': 0.002888},
  'rc': {'mean': 0.994979, 'std': 0.003639}},
 'MESH:D020211': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'ungrounded': {'f1': {'mean': 0.871429, 'std': 0.193781},
  'pr': {'mean': 0.816667, 'std': 0.260342},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'CHEBI:CHEBI:29191': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HGNC:348': {'f1': {'mean': 0.993184, 'std': 0.001279},
  'pr': {'mean': 0.994753, 'std': 0.003311},
  'rc': {'mean': 0.991645, 'std': 0.004123}}}

In [23]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [25]:
disamb.dump(model_name, results_path)

In [24]:
print(disamb.info())

Disambiguation model for AHR

Produces the disambiguations:
	AHR*	HGNC:348
	Autonomic Dysreflexia	MESH:D020211
	Respiratory Hypersensitivity*	MESH:D012130
	hydroxyl	CHEBI:CHEBI:29191

Class level metrics:
--------------------
Grounding                   	Count	F1     
Respiratory Hypersensitivity*	1385	 0.9964
                         AHR*	 952	0.99318
                  Ungrounded	  16	0.87143
       Autonomic Dysreflexia	   3	    0.4
                    hydroxyl	   3	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.99509
	Precision:	0.99362
	Recall:		0.99658

* Positive labels
See Docstring for explanation



In [26]:
model_to_s3(disamb)